# 1. Reranker应用
## FlagEmbedding

In [1]:
from FlagEmbedding import FlagReranker
MODEL_PATH = "/slurm/resources/weights/huggingface/BAAI/bge-reranker-large"

/slurm/home/admin/.conda/envs/dl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
reranker = FlagReranker(MODEL_PATH, use_fp16=True)

samples = [['什么是长城？', '你好'], 
           ['什么是长城？', '长城，又称万里长城，是中国古代的军事防御工程，是一道位于中国北部的长城。']]

scores = reranker.compute_score(samples)

print(scores)

[-7.625, 6.8515625]


## HuggingFace

In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

model.eval()

with torch.no_grad():
    inputs = tokenizer(samples, 
                       padding=True, 
                       truncation=True, 
                       return_tensors="pt", 
                       max_length=512)
    
    scores = model(**inputs, return_dict=True).logits.view(-1,).float()
    
    print(scores)

tensor([-7.6202,  6.8509])


## Reranker微调

这个重排序器基于[xlm-roberta-base](https://huggingface.co/FacebookAI/xlm-roberta-base)模型进行初始化，并使用了多个多语言数据集进行训练，主要包括中文、英文和其他语言的文本对。  

- 训练数据  
-- 中文数据集：总计788,491个文本对，来源于T2ranking、MMmarco、dulreader、Cmedqa-v2和nli-zh。  
-- 英文数据集：总计933,090个文本对，来源于msmarco、nq、hotpotqa和NLI。  
-- 其他语言数据集：总计97,458个文本对，涵盖阿拉伯语、孟加拉语、英语、芬兰语、印尼语、日语、韩语、俄语、斯瓦希里语、泰卢固语、泰语，数据来源于Mr.TyDi。   

- 跨语言检索数据集   
-- 为了增强跨语言检索能力，构建了两个基于MMarco的跨语言检索数据集：   
-- 从英文查询检索中文段落：随机抽样100,000个英文查询。  
-- 从中文查询检索英文段落：随机抽样100,000个中文查询。   
数据集发布于[Shitao/bge-reranker-data](https://huggingface.co/datasets/Shitao/bge-reranker-data/tree/main/data_v1)。   
 
- 模型性能  
-- 主要支持语言：中文和英文。   
-- 低资源语言性能：对于其他低资源语言，可能会出现性能下降的情况。  

###  微调数据格式  

与Embedding微调数据格式一致（难负例采样）  

```shell
{"query": "五个女人穿着人字拖沿着海滩走。", 
 "pos": ["一些穿着人字拖的女性正在海滩上行走"], 
 "neg": ["那个男人在谈论夏威夷。", "一名运动员正在参加1500米游泳比赛。", "战斗结束了。", "一个男人在山上滑雪。", "在公立学校，资金是我们的一个问题。", "我十八岁时面临一个严重的问题。", "一个穿着背心的男人坐在车里。", "乔治·布什告诉共和党人，他绝不会让他们考虑这个愚蠢的想法，违背了他顶级顾问的建议。"]}

{"query": "一个女人站在高崖上单腿站立，俯瞰一条河流。", 
 "pos": ["一个女人站在悬崖上。"], 
 "neg": ["一些穿着人字拖的女性正在海滩上行走", "那个男人在谈论夏威夷。", "我十八岁时面临一个严重的问题。", "他们因为退休而不是因为贷款而卖掉了他们的家。", "两个男性正在表演。", "她不会去法院清除自己的记录。", "孩子穿着黑色。", "有人密谋攻击Conrad，击中他的头部。"]}
```

### 开始训练

**1. 多块GPU的情况**  

```shell
torchrun --nproc_per_node {number of gpus} \
-m FlagEmbedding.reranker.run \
--output_dir /root/autodl-tmp/flagembedding/output_reranker_finetuned_model \
--model_name_or_path /root/autodl-tmp/models/bce-reranker-base_v1 \ # BAAI/bge-reranker-base
--train_data /root/autodl-tmp/samples/dataset_minedHN.jsonl \
--learning_rate 6e-5 \
--fp16 \
--num_train_epochs 5 \
--per_device_train_batch_size 1 \
--gradient_accumulation_steps 4 \
--dataloader_drop_last True \
--train_group_size 16 \
--max_len 512 \
--weight_decay 0.01 \
--logging_steps 10 
```  


**2. 单块GPU的情况**  

```shell
python -m FlagEmbedding.reranker.run \
--output_dir /root/autodl-tmp/flagembedding/output_reranker_finetuned_model \
--model_name_or_path /root/autodl-tmp/models/bce-reranker-base_v1 \
--train_data /root/autodl-tmp/samples/dataset_minedHN.jsonl \
--learning_rate 6e-5 \
--fp16 \
--num_train_epochs 5 \
--per_device_train_batch_size 1 \
--gradient_accumulation_steps 4 \
--dataloader_drop_last True \
--train_group_size 16 \
--max_len 512 \
--weight_decay 0.01 \
--logging_steps 10 
```

这里以“单块GPU的情况”为例，创建一个执行shell脚本：run_reranker_fintune.sh，将命令保存进去后执行脚本。

## 基于 LM-Cocktail 进行模型融合【可选操作】

在特定任务上微调基础模型（base bge model）可以提高其在目标任务上的性能，但可能会严重降低模型在目标领域之外的一般能力，例如在c-mteb任务上的性能下降。通过合并微调模型和基础模型，LM-Cocktail技术能够在提高下游任务性能的同时，保持在其他无关任务上的性能。

In [ ]:
from LM_Cocktail import mix_models, mix_models_with_data
model = mix_models(
    model_names_or_paths=["/root/autodl-tmp/models/bce-reranker-base_v1", "/root/autodl-tmp/flagembedding/output_reranker_finetuned_model"],
    model_type="reranker",
    weights=[0.5, 0.5],
    output_path="/root/autodl-tmp/flagembedding/output_reranker_merged_model"
)

## 加载微调后的模型进行测试
### 微调后的模型

In [ ]:
from FlagEmbedding import FlagReranker

MODEL_PATH = "/root/autodl-tmp/flagembedding/output_reranker_finetuned_model"

reranker = FlagReranker(MODEL_PATH, use_fp16=True)

samples = [['什么是长城？', '你好'], 
           ['什么是长城？', '长城，又称万里长城，是中国古代的军事防御工程，是一道位于中国北部的长城。']]

scores = reranker.compute_score(samples)

print(scores)

###  融合模型

In [ ]:
from FlagEmbedding import FlagReranker

MODEL_PATH = "/root/autodl-tmp/flagembedding/output_reranker_merged_model"

reranker = FlagReranker(MODEL_PATH, use_fp16=True)

samples = [['什么是长城？', '你好'], 
           ['什么是长城？', '长城，又称万里长城，是中国古代的军事防御工程，是一道位于中国北部的长城。']]

scores = reranker.compute_score(samples)

print(scores)

## reranker模型评估

安装 C-MTEB（Chinese Massive Text Embedding Benchmark）:  

pip install -U C_MTEB  

或   

git clone https://github.com/FlagOpen/FlagEmbedding.git  
cd FlagEmbedding/C_MTEB  
pip install -e .  

**对 reranker 进行评估**  

```shell
python eval_cross_encoder.py --model_name_or_path /root/autodl-tmp/models/bce-reranker-base_v1
```